In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import plotly.io as pio
import pandas as pd
import requests
import zipcodes
from pprint import pprint
from sklearn.preprocessing import scale
from sklearn import preprocessing
import os

min_max_scaler = preprocessing.MinMaxScaler()

In [2]:

def lat(row):
    zips = zipcodes.matching(row['Mail Zipcode'])
    for zip_code in zips:
        if zip_code['city'] == "SAN FRANCISCO":
            return str(zip_code['lat'])
        
def long(row):
    zips = zipcodes.matching(row['Mail Zipcode'])
    for zip_code in zips:
        if zip_code['city'] == "SAN FRANCISCO":
            return str(zip_code['long'])
        



df = pd.read_csv("/Users/nigel/SFPropertyTax/sf_new_business_neighborhood_price_appreciation.csv")
df.head()
df['Mail Zipcode'] = df['Mail Zipcode'].astype(str)
df['lat'] = df.apply(lat, axis=1)
df['long'] = df.apply(long, axis=1)
df['scaled new business count'] = scale(df['new business count'])
df['scaled pct change'] = scale(df['pct_change'])
df.head()
# df = df[(df['pct_change'] < 1) & (df['pct_change'] > -1)] 

,Unnamed: 0,ROLLYEAR,Mail Zipcode,new business count,pct_change,median land value price,lat,long,scaled new business count,scaled pct change
0,152,2014,94110,418.0,0.062268,242810.0,37.74,-122.41,3.571636,-0.044821
1,151,2013,94110,399.0,0.092849,228577.0,37.74,-122.41,3.368907,0.167473
2,109,2013,94103,396.0,0.068476,302853.0,37.77,-122.41,3.336897,-0.001727
3,150,2012,94110,369.0,0.104932,209157.0,37.74,-122.41,3.048809,0.251355
4,130,2013,94107,361.0,0.118523,313200.0,37.76,-122.39,2.963449,0.345707


In [3]:
years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

In [5]:
import plotly.plotly as py
import plotly.graph_objs as go

if not os.path.exists('mapcharts'):
    os.mkdir('mapcharts')


plotly.tools.set_credentials_file(username='graphdude', api_key='Cg2SFmNYy4XfTaGHTHro')

mapbox_access_token = 'pk.eyJ1IjoibmlnZWxoYWxsMSIsImEiOiJjanc2enhqZ3Axb3pvNDRwY3Rlam5rdmpjIn0.Mm898m2mXrYY0K-2JxStEg'

plots = []

# def plot_year(year):

year = 2010

df1 = df[df['ROLLYEAR'] == year]



for i, row in df1.iterrows():
#     print(row['lat'])
    plots.append(go.Scattermapbox(
        lat=[row['lat']],
        lon=[row['long']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color = 'blue',
            opacity=.5,
            size=(1+row['scaled new business count'])*10
        ),
        name=row['Mail Zipcode'],
        text=['San Francisco'],
    ))

for i, row in df1.iterrows():

    size = (1+row['pct_change'])**20
    color = 'green' if row['pct_change'] > 0 else "red"
#     print(row['lat'])
    plots.append(go.Scattermapbox(
        lat=[row['lat']],
        lon=[row['long']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color = color,
            opacity=.5,
            size=20 if size > 10 else size
        ),
        text=['San Francisco'],
    ))


layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=37.74,
            lon=-122.4241
        ),
        pitch=0,
        zoom=10.50
    ),
)

print(df1.sort_values("pct_change", ascending=False).head())
fig = go.Figure(data=plots, layout=layout)
py.iplot(fig, filename='New Business and Appreciation {} Mapbox'.format(year))
# pio.write_image(fig, 'mapcharts/{}.png'.format(year))

     Unnamed: 0  ROLLYEAR Mail Zipcode  new business count  pct_change  \
208         267      2010        94158                12.0    1.879103   
64          113      2010        94104               167.0    0.288957   
287         280      2010        94402                 4.0    0.204972   
325         331      2010        94904                 1.0    0.201272   
279         272      2010        94159                 5.0    0.161406   

     median land value price    lat     long  scaled new business count  \
208                 368125.0  37.76  -122.39                  -0.760360   
64                  449395.0  37.79   -122.4                   0.893481   
287                 118656.0   None     None                  -0.845720   
325                 118049.0   None     None                  -0.877730   
279                 238137.0  37.78  -122.45                  -0.835050   

     scaled pct change  
208          12.567714  
64            1.528866  
287           0.945836  
325 

In [ ]:
plot_year(2010)